In [1]:
import pandas as pd
import json
import requests
import numpy as np
import folium
import IPython

carto_acc='wri-01'
sql_endpoint='https://{0}.carto.com/api/v2/sql'.format(carto_acc)
map_endpoint='https://{0}.carto.com/api/v1/map'.format(carto_acc)

## Data layers structure 

### Data table

In [2]:
payload = {'q': "SELECT * FROM water_risk_indicators limit 1"}
r = requests.get(sql_endpoint, params=payload)

tableStructure= pd.read_json(json.dumps(r.json()['rows']), orient='records')
tableStructure.head()

,basinid,cartodb_id,indicator,label,model,period,period_value,the_geom,the_geom_webmercator,type,value,year
0,4774,451,water_stress,NaN,historic,year,0,NaN,NaN,absolute,0,2014


### Geometry table

#### WWF hydrobasins level 6:
Used for aqueduct v3.x the id used is PFAF_ID  that correspond to [Pfafstetter Coding System](https://en.wikipedia.org/wiki/Pfafstetter_Coding_System) 

In [ ]:

iframe = '<iframe width="100%" height="520" frameborder="0" src="https://resourcewatch.carto.com/u/wri-rw/builder/db3c53be-4787-11e7-9eb1-0ee66e2c9693/embed" allowfullscreen webkitallowfullscreen mozallowfullscreen oallowfullscreen msallowfullscreen></iframe>'
IPython.display.HTML(iframe)

#### GDBD:

Currently used for aqueduct v2.x; on the platform they will cover projected data until we recived v3 projected data.

In [ ]:
payload = {'q': "SELECT * FROM wri_subcatchements Limit 1"}
r = requests.get(sql_endpoint, params=payload)

tableStructure= pd.read_json(json.dumps(r.json()['rows']), orient='records')
tableStructure.head(0)

**Key column for joins:** basinid

Parameters definition:  

* **{{indicator}}:**  
    [Indicator list](#cell2)


* **{{scenario}}:**  
    historic ( bs )  
    pessimistic ( 38 ) ssp3 rcp85  
    optimistic ( 24 ) ssp2 rcp45  
    bau ( 28 ) ssp2 rcp85 
 
 
* **{{period_type}}:**  
    month,  
    year,   
    quarter 
    
    
* **{{period_value}}:**  
    month: 1:12   
    year: null  
    quarter: 1:4
    
    
* **{{year}}:**  
    2014   
    2020   
    2030   
    2040  
    
    
* **{{data_type}}:**  
    absolute  
    change from baseline  
    uncertainity

#### Raster layers 
The table naming convention will follow this structure:
```
{{indicator}}-{{scenario}}-{{period_type}}-{{year}}-{{period_value}}.tif
``` 
example:  
drought_severity_flow-historic-year-2014.tif

#### Example geometry map

```sql
with r as (SELECT basinid, label, value FROM water_risk_indicators where indicator = 'water_stress' and model = 'historic' and period = 'year' and period_value is null and year =2014 and type = 'absolute')

select s.*, label, value from wri_subcatchements s left join r on s.basinid=r.basinid

```

``` css
#water_risk_indicators{
  polygon-fill:transparent;
  polygon-opacity: 1;
  line-color:transparent;
  line-width: 1;
  line-opacity: 1;
}

#water_risk_indicators [ label='5. Extremely high (>80%)'] {
   polygon-fill:#B10026;
  line-color:#B10026
}
#water_risk_indicators [ label='4. High (40-80%)'] {
   polygon-fill:  #FC4E2A;
  line-color:  #FC4E2A
}
#water_risk_indicators [ label='3. Medium to high (20-40%)'] {
   polygon-fill: #FD8D3C;
  line-color: #FD8D3C
}
#water_risk_indicators [ label='2. Low to medium (10-20%)'] {
   polygon-fill: #FED976;
  line-color:  #FED976
}
#water_risk_indicators [label='1. Low (<10%)'] {
   polygon-fill: #FFFFB2;
  line-color:  #FFFFB2
}

#water_risk_indicators [label='Arid & low water use'] {
   polygon-fill: #eee;
  line-color:  #eee
}
```

In [ ]:
api_endpoint = 

#### Example raster map

```sql
select * from drought_severity_flow-historic-year-2014

```

``` css
#water_risk_indicators{
  raster-opacity:1;
  raster-scaling:near;
  raster-colorizer-default-mode: exact;
  raster-colorizer-default-color:  transparent;
  raster-colorizer-epsilon:0.11;
  raster-colorizer-stops:
    stop(0, #eee) /* Arid & low water use */ 
    stop(1, #D2A965) /* 1. Low */
    stop(2, #FED976) /* 2. Low to medium */
    stop(3, #FD8D3C) /* 3. Medium to high */
    stop(4, #FC4E2A) /* 4. High */
    stop(5, #B10026) /* 5. Extremely high */
}
``` 

<a id="cell2"></a>

## Old indicators

| Family |Indicator name| slug |
|---|---|--- |
| Physical Risk Quantity |Baseline Water Stress  | water_stress |
| Physical Risk Quantity |Interannual Variability  |  interannual_variability |
| Physical Risk Quantity |Seasonal Variability  | seasonal_variability  |
| Physical Risk Quantity |Flood Occurrence  |  flood_occurrence |
| Physical Risk Quantity |Drought Severity  | drought_severity  |
| Physical Risk Quantity |Upstream Storage  | upstream_storage |
| Physical Risk Quantity |Groundwater Stress  | groundwater_stress|
| Physical Risk Quality |Return Flow Ratio  | return_flow_ratio |
| Physical Risk Quality |Upstream Protected Land  | upstream_protected_land |
| Regulatory & Reputational Risk |Media Coverage   |media_coverage  |
| Regulatory & Reputational Risk |Access to Water   |access_to_water  |
| Regulatory & Reputational Risk |Threatened Amphibians  |threatened_amphibians|

### Old indicators correspondence with new ones
[Indicator list reference](https://docs.google.com/spreadsheets/d/11GWSY4HZdskCFoVpGCTmzMw8evifIZfHDi6_UaoGQFE/edit#gid=0)

|New Indicators|Old Indicators slug |
|---|--- |
|Baseline Water Stress && Future Water Stress  |water_stress |
|Inter Annual Variability  |  interannual_variability |
|Seasonal variability && Projected Change in Seasonal variability  | seasonal_variability  |
|Drought Severity  | drought_severity  |
|Groundwater Stress  | groundwater_stress  |
|Groundwater Table Declining Trend  | -
|Probability of inundation from river floods  |  -
|Index for Coastal eutrophication Potential (ICEP)  | -
|Reprisk Index (RRI)  | -
|Threatened Amphibians  |threatened_amphibians  |
|Access to water  |access_to_water  |
|Flood Occurrence  |flood_occurrence|

### Not included in the new ones
upstream_protected_land  
upstream_storage  
return_flow_ratio   
water_demand  
media_coverage  
water_supply  

### AQ.v3 indicators
| Family |Indicator name|slug|visual resolution|temporal resolution|
|---|---|---|---|---|
| Physical Risk Quantity |Baseline Water Stress  | water_stress |Sub-basin | year, month |
| Physical Risk Quantity |Interannual Variability  |  interannual_variability |Pixel | year, month |
| Physical Risk Quantity |Seasonal Variability  | seasonal_variability  |Pixel | year |
| Physical Risk Quantity |Flood Occurrence  |  flood_occurrence | - | - |
| Physical Risk Quantity |Drought Severity  | drought_severity  | Pixel |year, month |
| Physical Risk Quantity |Groundwater Stress  | groundwater_stress| Aquifer | year |
| Physical Risk Quality |Primary waste water treatment  | p_waste_w_ratio | Country | year |
| Physical Risk Quality |Indicator of Coastal Eutrophication Potential (ICEP)  | icep | Sub-Basin | year |
| Regulatory & Reputational Risk |RepRisk Index   |media_coverage  | Country | year |
| Regulatory & Reputational Risk |Access to Improved Drinking Water   |access_drink_water  | Country | year |
| Regulatory & Reputational Risk |IUCN Red List of Threatened Freshwater Species  |threatened_freshw_species| Pixel | year |